In [30]:

with open("scaler.pkl", "rb") as scaler_file:
    scaler = pickle.load(scaler_file)


with open("rf_model.pkl", "rb") as model_file:
    model = pickle.load(model_file)

In [35]:
import json
import numpy as np

def predict_game_winner(team_0_starters, team_1_starters, model, league_file='league.json'):
    """
    Predict the winner of a game given the 5 starters for each team.
    
    Args:
        team_0_starters (list): List of 5 player names for Team 0.
        team_1_starters (list): List of 5 player names for Team 1.
        model (sklearn model): Trained random forest model.
        league_file (str): Path to the league.json file containing player attributes.
    
    Returns:
        int: 0 if Team 0 is predicted to win, 1 if Team 1 is predicted to win.
    """
    # Load the league.json data
    with open(league_file, 'r') as f:
        player_data = json.load(f)
    

    # Convert player data to a dictionary for faster lookup
    player_dict = {player['name']: player for player in player_data}
  
    
    # Compute the average attributes for both teams
    team_0_avg = compute_team_average(team_0_starters, player_dict)
    team_1_avg = compute_team_average(team_1_starters, player_dict)
    
    # Combine features for both teams into a single input for the model
    combined_features = pd.concat([team_0_avg.add_prefix('team_0_'), team_1_avg.add_prefix('team_1_')]).to_frame().T
    
    # Scale the combined features using the same scaler used during training
    scaled_features = scaler.transform(combined_features)
    
    # Predict the winner
    prediction = model.predict(scaled_features)
    
    return prediction[0]  # 0 for Team 0 win, 1 for Team 1 win

# Helper function to compute average attributes for a team
def compute_team_average(starters, player_dict):
    attributes_list = []
    for player in starters:
        if player in player_dict:
            attributes_list.append(player_dict[player])
        else:
            raise ValueError(f"Player '{player}' not found in league.json")
    # Convert to a DataFrame and compute the mean of all attributes
    return pd.DataFrame(attributes_list).mean()


In [36]:
team_0_starters = ["Luka Doncic", "Stephen Curry", "Jayson Tatum", "Karl-Anthony Towns", "Nikola Jokic"]
team_1_starters = ["LeBron James", "Kevin Durant", "Anthony Edwards", "Victor Wembanyama", "Joel Embiid"]

prediction = predict_game_winner(team_0_starters, team_1_starters, model, league_file='league.json')

if prediction == 0:
    print("Team 0 is predicted to win.")
else:
    print("Team 1 is predicted to win.")


Team 0 is predicted to win.


/var/folders/gv/mxw7p1nn5_b2n6n_7wkjz2sw0000gn/T/ipykernel_51356/1275881089.py:50: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  return pd.DataFrame(attributes_list).mean()
/var/folders/gv/mxw7p1nn5_b2n6n_7wkjz2sw0000gn/T/ipykernel_51356/1275881089.py:50: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  return pd.DataFrame(attributes_list).mean()
